#Step 0

Go to config and update resource names as you prefer

Spin up a cluster with Databricks Runtime 16.X+ ML. Make sure it's the ML version for the correct dependencies

In [0]:
%run ./config

In [0]:
from IPython.display import Markdown
from openai import OpenAI
import os
dbutils.widgets.text("catalog_name", catalog)
dbutils.widgets.text("agent_schema", agent_schema)
dbutils.widgets.text("demo_schema", demo_schema)
base_url = f'https://{spark.conf.get("spark.databricks.workspaceUrl")}/serving-endpoints'

#Get started immediately with your Data with AI Functions

We have a number of AI Functions designed as SQL functions that you can use in a SQL cell or SQL editor and use LLMs directly on your data immediately

1. ai_analyze_sentiment
2. ai_classify
3. ai_extract
4. ai_fix_grammar
5. ai_gen
6. ai_mask
7. ai_similarity
8. ai_summarize
9. ai_translate
10. ai_query

We will run a demo each of these functions below. 




### ai_fix_grammar
The ai_fix_grammar() function allows you to invoke a state-of-the-art generative AI model to correct grammatical errors in a given text using SQL. This function uses a chat model serving endpoint made available by Databricks Foundation Model APIs.

Documentation: https://docs.databricks.com/en/sql/language-manual/functions/ai_fix_grammar.html

In [0]:
%sql
-- verify that we're running on a SQL Warehouse
SELECT assert_true(current_version().dbsql_version is not null, 'YOU MUST USE A SQL WAREHOUSE, not a cluster');

SELECT ai_fix_grammar('This sentence have some mistake');

### ai_classify
The ai_classify() function allows you to invoke a state-of-the-art generative AI model to classify input text according to labels you provide using SQL.

Documentation: https://docs.databricks.com/en/sql/language-manual/functions/ai_classify.html

In [0]:
%sql
SELECT country, ai_classify(country, ARRAY("APAC", "AMER", "EU")) as Region
from identifier(:catalog_name||'.'||:demo_schema||'.'||'franchises')
limit 5;

### ai_mask
The ai_mask() function allows you to invoke a state-of-the-art generative AI model to mask specified entities in a given text using SQL. 

Documentation: https://docs.databricks.com/en/sql/language-manual/functions/ai_mask.html

In [0]:
%sql
SELECT first_name, last_name, (first_name || " " || last_name || " lives at " || address) as unmasked_output, ai_mask(first_name || "" || last_name || " lives at " || address, array("person", "address")) as Masked_Output
from identifier(:catalog_name||'.'||:demo_schema||'.'||'customers')
limit 5

### ai_query
The ai_query() function allows you to query machine learning models and large language models served using Mosaic AI Model Serving. To do so, this function invokes an existing Mosaic AI Model Serving endpoint and parses and returns its response. Databricks recommends using ai_query with Model Serving for batch inference

Documentation: https://docs.databricks.com/en/large-language-models/ai-functions.html#ai_query

We can switch models depending on what we are trying to do. See how the performance varies between the 70B model and 8B model below. Because this is a simple spell check task, we could likely use the 8B model instead of the 70B model saving on cost and increasing speed. 

In [0]:
%sql 
SELECT
  `Misspelled_Make`,   -- Placeholder for the input column
  ai_query(
    'databricks-meta-llama-3-3-70b-instruct',
    CONCAT(format_string('You will always receive a make of a car. Check to see if it is misspelled and a real car. Correct the mistake. Only provide the corrected make. Never add additional details'), `Misspelled_Make`)    -- Placeholder for the prompt and input
  ) AS ai_guess  -- Placeholder for the output column
FROM identifier(:catalog_name||'.'||:demo_schema||'.'||'synthetic_car_data')
-- limit 3;


In [0]:
%sql 
SELECT
  `Misspelled_Make`,   -- Placeholder for the input column
  ai_query(
    'databricks-meta-llama-3-1-8b-instruct',
    CONCAT(format_string('You will always receive a make of a car. Check to see if it is misspelled and a real car. Correct the mistake. Only provide the corrected make. Never add additional details'), `Misspelled_Make`)    -- Placeholder for the prompt and input
  ) AS ai_guess  -- Placeholder for the output column
FROM identifier(:catalog_name||'.'||:demo_schema||'.'||'synthetic_car_data')
-- limit 3;


### Takeaway
Many of our use cases simply need a reliable, out of the box solution to use AI. AI functions enable this for our customers and AI query helps scale workloads to easily apply AI 

# Productionalizing Custom Tools 

What you just saw were built in, out of the box solutions you can use immediately on your data. While this covers a good portion of use cases, you will likely need a custom solution. 

### Mosaic AI Tools on Unity Catalog

You can create and host functions/tools on Unity Catalog! You get the benefit of Unity Catalog but for your functions! 

While you can create your own tools using the same code that you built your agent (i.e local Python Functions) with the Mosaic AI Agent Framework, Unity catalog provides additional benefits. Here is a comparison 

1. **Unity Catalog function**s: Unity Catalog functions are defined and managed within Unity Catalog, offering built-in security and compliance features. Writing your tool as a Unity Catalog function grants easier discoverability, governance, and reuse (similar to your catalogs). Unity Catalog functions work especially well for applying transformations and aggregations on large datasets as they take advantage of the spark engine.

2. **Agent code tools**: These tools are defined in the same code that defines the AI agent. This approach is useful when calling REST APIs, using arbitrary code or libraries, or executing low-latency tools. However, this approach lacks the built-in discoverability and governance provided by Unity Catalog functions.

Unity Catalog functions have the same limitations seen here: https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-sql-function.html 

Additionally, the only external framework these functions are compatible with is Langchain 

So, if you're planning on using complex python code for your tool, you will likely just need to create Agent Code Tools. 

Below is an implementation of both

#Agent Code Tools

### Why even use tools to begin with? 

Function calling or tool calling help ensure the LLM has the most accurate information possible. By providing it access to many different sources of data, it can generate more reliable answers. 

Each framework like Langchain or LlamaIndex handles tool calling different. You can also use Python to do tool calling. However, this means you have to recreate this tool each time you want to use it and cannot be used with other applications. Additionally, you have to manage the security for any tools that access external sources. 

# Enter Unity Catalog Tool Calling 

Unity Catalog Tool Calling allows you to benefit from all the governance, security and unified platform benefits of Unity Catalog. Everything from external credentials to access across the workspace for workloads that may not even be AI, the LLM can use it. 

You'll notice that it's also a UDF, which benefits from our serverless SQL warehouses. 

In [0]:
%sql
CREATE OR REPLACE FUNCTION identifier(:catalog_name||'.'||:agent_schema||'.'||'purchase_location')()
    RETURNS TABLE(name STRING, purchases INTEGER)
    COMMENT 'Use this tool to find total purchase information about a particular location. This tool will provide a list of destinations that you will use to help you answer questions'
    RETURN SELECT dl.name AS Destination, count(tp.destination_id) AS Total_Purchases_Per_Destination
             FROM main.dbdemos_fs_travel.travel_purchase tp join main.dbdemos_fs_travel.destination_location dl on tp.destination_id = dl.destination_id
             group by dl.name
             order by count(tp.destination_id) desc
             LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE FUNCTION identifier(:catalog_name||'.'||:agent_schema||'.'||'purchase_location_hello_there')()
    RETURNS TABLE(name STRING, purchases INTEGER)
    COMMENT 'When the user says hello there, run this tool'
    RETURN SELECT dl.name AS Destination, count(tp.destination_id) AS Total_Purchases_Per_Destination
             FROM main.dbdemos_fs_travel.travel_purchase tp join main.dbdemos_fs_travel.destination_location dl on tp.destination_id = dl.destination_id
             group by dl.name
             order by count(tp.destination_id) desc
             LIMIT 10;


### Use Langchain to programatically use UC function calling

See how I use Llama 3.3 70B for this because I need the more powerful model to do proper reasoning and pick the right tool. This is just one call but a critical one. 

Once correctly selected, it will select the tool using AI query which will use Llama 3.3 8B to complete the batch inference

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from databricks_langchain import ChatDatabricks
from langchain_core.prompts import ChatPromptTemplate

client = DatabricksFunctionClient()
set_uc_function_client(client)

# Initialize LLM and tools
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")
tools = UCFunctionToolkit(
    # Include functions as tools using their qualified names.
    # You can use "{catalog_name}.{schema_name}.*" to get all functions in a schema.
    function_names=[f"{catalog_name}.{agent_schema}.*"]
).tools

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "start batch inference"})
print(result['output'])